In [1]:
import os

In [2]:
%pwd

'e:\\MLprojects\\electricitybill\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\MLprojects\\electricitybill'

In [5]:
from dataclasses import dataclass 
from pathlib import Path 


@dataclass()
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path 
    unzip_dir: Path

In [6]:
from src.ElectricityBill.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH, SCHEMA_FILE_PATH
from src.ElectricityBill.utils.commons import read_yaml, create_directories


In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
        schema_filepath=SCHEMA_FILE_PATH):

        
        """Initialize ConfigurationManager."""
        # Read YAML configuration files to initialize configuration parameters
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        # Create necessary directories specified in the configuration
        create_directories([self.config.artifacts_root])

    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        """Get data ingestion configuration."""
        # Get data ingestion section from config
        config = self.config.data_ingestion

        # Create DataIngestionConfig object
        create_directories([config.root_dir])

        # Create and return DataIngestionConfig object
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir,
        )

        return data_ingestion_config



In [8]:
import os
import urllib.request as request
import zipfile
from src.ElectricityBill import logging
from src.ElectricityBill.utils.commons import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logging.info(f"{filename} download! with following info: \n{headers}")
        else:
            logging.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
    logging.info("Data Ingestion Completed!")
except Exception as e:
    raise e


[2024-04-28 01:29:33,470 ] 32 root - INFO - yaml file: config\config.yaml loaded successfully
{'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://github.com/minich-code/datahub/raw/main/Electricity%20Bill%20Pred%20data.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}}
[2024-04-28 01:29:33,492 ] 32 root - INFO - yaml file: params.yaml loaded successfully
{'key': 'val'}
[2024-04-28 01:29:33,496 ] 32 root - INFO - yaml file: schema.yaml loaded successfully
{'COLUMNS': {'Fan': 'int64', 'Refrigerator': 'float64', 'AirConditioner': 'float64', 'Television': 'float64', 'Monitor': 'float64', 'MotorPump': 'int64', 'Month': 'int64', 'City': 'object', 'Company': 'object', 'MonthlyHours': 'int64', 'TariffRate': 'float64', 'ElectricityBill': 'float64'}, 'TARGET_COLUMN': {'name': 'ElectricityBill'}}
[2024-04-28 01:29:33,498 ] 53 root - INFO - created directory at: artifacts
[2024-04-28 